# Cel
Wyciągnięcie danych na temat dróg i POI w województwie śląskim.

In [1]:
import json
import glob

import geopandas as gpd
import numpy as np
import pandas as pd
from OSMPythonTools.nominatim import Nominatim
from tqdm import tqdm

In [2]:
%%writefile overpass.py

import urllib

from OSMPythonTools.overpass import Overpass


class CustomOverpass(Overpass):
    
    # make sure it is GET method with properly (from the point of overpass api server) 
    # escaped query strings
    def _queryRequest(self, endpoint, queryString, params=None):
        # is the the overpass server broken?
        # qstr = urllib.parse.urlencode({'data': queryString, **(params or {})})
        qstr = "&".join(f"{k}={v}" for k, v in {'data': queryString, **(params or {})}.items())
        print( endpoint + f"interpreter?{qstr}")
        return urllib.request.Request(
            endpoint + f"interpreter?{qstr}", 
            method="GET"
        )
    
    # add util method for building sample queries
    def from_statements(self, statements, **kwargs):
        query = (
            "(" + 
            ";".join(statement for statement in statements) +
            ";);" +
            "out+skel+body+meta+geom;"
        )
        
        return self.query(query, **kwargs)

Overwriting overpass.py


In [3]:
from overpass import *
nominatim = Nominatim()          
overpass = CustomOverpass()

In [4]:
silesia = nominatim.query("województwo śląskie", params={"polygon_geojson": 1}).toJSON()[0]
bbox = silesia["boundingbox"]

In [5]:
%%time
# get road segments with all possible tags
road_segments = overpass.from_statements(
    statements = [
        "way[highway](bbox)",
        "node(w)",
    ],
    params={"bbox": ",".join([bbox[2], bbox[0], bbox[3], bbox[1]])},
    timeout=3600
)

CPU times: user 47.6 s, sys: 19.7 s, total: 1min 7s
Wall time: 1min 14s


In [6]:
def build_feature(element, coords, shape_type, values_to_ignore=None):
    values_to_ignore = values_to_ignore or []
    tags = {
        k: v
        for k, v in element.get("tags", {}).items()
        if v not in values_to_ignore
    }
    
    return {
        "type": "Feature",
        "id": f"{element['type']}/{element['id']}",
        "properties": {
            "id": f"{element['type']}/{element['id']}",
            **tags
        },
        "geometry": {
            "type": shape_type,
            "coordinates": coords
        }
    }


def transform_to_geojson(overpass_elements, types=None, values_to_ignore=None, **kwargs):
    if not types:
        types = []
    
    nodes = [element for element in overpass_elements if element["type"] == "node"]
    ways = [element for element in overpass_elements if element["type"] == "way"]
    relations = [element for element in overpass_elements if element["type"] == "relation"]
    
    return {
        "type": "FeatureCollection",
        "features": (
            ([
                build_feature(
                    element, 
                    (element["lon"], element["lat"]),
                    shape_type="Point",
                    values_to_ignore=values_to_ignore
                )
                for element in nodes if kwargs.get("node_condition", lambda x: x)(element)
            ] if "nodes" in types else [])
            + ([
                build_feature(
                    element, 
                    [(node["lon"], node["lat"]) for node in element["geometry"]],
                    shape_type="LineString",
                    values_to_ignore=values_to_ignore
                )
                for element in ways            
            ] if "ways" in types else [])
            + ([
                build_feature(
                    element, 
                    [
                        tuple([
                            (point["lon"], point["lat"]) 
                            for point in node["geometry"]
                        ])
                        for node in element["members"]
                        if node["type"] == "way"
                    ],
                    shape_type="MultiLineString",
                    values_to_ignore=values_to_ignore
                )
                for element in relations 
            ] if "relations" in types else [])
        )
    }

In [7]:
%%time

def batch_entries(collection, batch_size=25000):
    ix = 0

    while collection[ix * batch_size: (ix + 1) * batch_size ]:
        yield collection[ix * batch_size: (ix + 1) * batch_size]
        ix += 1


ix = 0
for collection in tqdm(batch_entries(road_segments.toJSON()["elements"])):
    ways_data = transform_to_geojson(
        collection, 
        types=["ways"],
        values_to_ignore=["2017-09-31"]
    )
    if ways_data["features"]:
        with open(f"../output/ways.{ix:04}.geojson", "w") as f:
            f.write(json.dumps(ways_data))
    
    nodes_data = transform_to_geojson(
        collection, 
        types=["nodes"],
        node_condition=lambda x: x.get("tags")
    )
    if nodes_data["features"]:
        with open(f"../output/nodes.{ix:04}.geojson", "w") as f:
            f.write(json.dumps(nodes_data))
    
    ix += 1

178it [00:31,  5.64it/s]

CPU times: user 20.6 s, sys: 6.95 s, total: 27.5 s
Wall time: 31.7 s


In [8]:
%%time
from collections import Counter

def find_intersections(overpass_elements):
    counter = Counter()
    nodes = {
        element["id"]: (element["lat"], element["lon"])
        for element in overpass_elements if element["type"] == "node"
    }
    highways = {
        "motorway", 
        "trunk",
        "primary",
        "secondary",
        "tertiary",
        "unclassified",
        "residential",
        "service",
        "motorway_link",
        "trunk_link",
        "primary_link",
        "secondary_link",
        "motorway_junction"
    }
    for way in overpass_elements:
        if way["type"] == "way" and way["tags"]["highway"] in highways:
            for node in way["nodes"]:
                counter[node] += 1

    return [nodes[node_id] for (node_id, count) in counter.items() if count > 1]


df = pd.DataFrame.from_records(
    find_intersections(road_segments.toJSON()["elements"]),
    columns=("lat", "lon")
)
intersections_gdf = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(df["lon"], df["lat"]), 
    crs="epsg:4326",
)
del df

CPU times: user 13.1 s, sys: 11.1 s, total: 24.2 s
Wall time: 29.1 s


In [9]:
%%time
shape_gdf = gpd.read_file(json.dumps(silesia))

def build_dataframe(pattern):
    for file in tqdm(sorted(glob.glob(pattern))):
        yield gpd.sjoin(
            gpd.read_file(file, crs="epsg:4326"), 
            shape_gdf, 
            op="within"
        )

roads_gdfs = list(build_dataframe(pattern="../output/ways*.geojson"))
pois_gdfs = list(build_dataframe(pattern="../output/nodes*.geojson"))


100%|██████████| 152/152 [00:29<00:00,  5.13it/s]

CPU times: user 5min 17s, sys: 7.91 s, total: 5min 25s
Wall time: 5min 33s


In [10]:
%%time
roads = pd.concat(roads_gdfs).replace({None: np.nan})
pois = pd.concat(pois_gdfs).replace({None: np.nan})

CPU times: user 1min 39s, sys: 29.9 s, total: 2min 9s
Wall time: 2min 21s


In [11]:
%%time

import pickle

def save_pickle(fname, data):
    with open(f"../output/{fname}.pickle", "wb") as f:
        pickle.dump(data, f, protocol=4)

def read_pickle(fname):
    with open(f"../input/{fname}.pickle", "rb") as f:
        return pickle.load(f)


save_pickle("roads", roads)
save_pickle("pois", pois)
save_pickle("intersections", intersections_gdf)

CPU times: user 18.7 s, sys: 3.95 s, total: 22.7 s
Wall time: 26.8 s
